# listing movies from bollyshare

In [ ]:
import requests, bs4 ,pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

url ='https://freemoviewap2019.com/hollywood_dub_movie{0}.php'

movieCollections = pd.DataFrame({
        'Title' : [] ,
        'Url' : []
    });

session = requests.Session()

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('https://', HTTPAdapter(max_retries=retries))


def SearchAllPages():
    global movieCollections
    global url  
    status =200
    pageNo =1    
    while(pageNo<=100): #status !=404 and
        new_url=url.format( 's_'+ str(pageNo) if pageNo>1 else '')
        req= session.get(new_url)       
        status= req.status_code     
        print('{0} with status {1}'.format(new_url,status) )
        if(status ==200):
            rs = readPage(req.text)
            movieCollections =  movieCollections.append( pd.DataFrame(rs))
        pageNo+=1
        
            
 

def readPage(text):
        data= bs4.BeautifulSoup(text,'html5lib')
        listitems= data.select('.w3_agile_featured_movies > .w3l-movie-gride-agile ')
        titles =[item.select_one('.w3l-movie-text a').get_text() for item in listitems]
        urls = [item.select_one('.w3l-movie-text a').get('href') for item in listitems]
        return { 'Title' : titles , 'Url' : urls };
    
        
        
SearchAllPages()

movieCollections.to_csv('bollyshare_movies.csv')

In [36]:
import requests, bs4 ,re,urllib, pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import progressbar
from time import sleep

data = pd.read_csv("bollyshare_movies_final.csv") 
imdb_path='https://www.imdb.com/search/title/?title={0}&adult=include&view=simple&count=1&year={1}'


def getProgressBar(max):
    return  progressbar.ProgressBar(maxval=max,widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    
    

def getSession(retry=5):
    session = requests.Session()
    retries = Retry(total=retry,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    session.mount('http://', HTTPAdapter(max_retries=retries))
    return session


def getImdbRank(info):
    try:
        global imdb_path
        session=getSession() 
        title = urllib.parse.quote_plus(info.get('title')) 
        url =imdb_path.format(title ,info.get('year'))
        data=session.get(url)
        data= bs4.BeautifulSoup(data.text,'html5lib')
        itemDataText= data.select_one('div.lister-item .lister-item-content')  
        if(itemDataText is not None):
            return {
            'title':itemDataText.select_one('.col-title span.lister-item-header span[title] a').get_text().strip(),
            'rating': itemDataText.select_one('.col-imdb-rating').get_text().strip(),
            'link':url
            }
    except Exception as e:
        print('getImdbRank ',e ,type(e).__name__)   
    return {}


def FillMetaData():
        metaCollections = pd.DataFrame();
        bar=getProgressBar(len(data.index))
        bar.start()
        session=getSession() 
        progress=1
        for _,d in data.iterrows():
            try:
                req =session.get(d.Url)
                status= req.status_code  
                if(status ==200):
                    rs= readPage(req.text)
                    rs.update({'Url':d.Url})
                    metaCollections=metaCollections.append(pd.DataFrame.from_dict([rs]))
            except Exception as e:
                print('FillMetaData',e,type(e).__name__)       
            finally :
                bar.update(progress)
                progress+=1
                
        bar.finish()
        return metaCollections
           
           
        
def readPage(text):
    data= bs4.BeautifulSoup(text,'html5lib')
    itemDataText= data.select('.list-group  a.list-group-item ')
    itemData ={ itemText.text.split()[-1]:itemText.select_one('span.badge').get_text()  for itemText in itemDataText if itemText is not None }
    
    download_links={ 'Download Full Movie':data.find(lambda tag:tag.name=="a" and "Download Full Movie" in tag.text),
                     'Download Full HD':data.find(lambda tag:tag.name=="a" and "Download Full HD" in tag.text)  }
    
    itemData.update({key : value.get('href') for key, value in download_links.items() if value is not None })
    itemData.update({'Year' : re.sub('^.*\((.*?)\)[^\(]*$', '\g<1>', itemData.get("Movie")) ,
                     'Movie': re.sub(r'\(.*?\)', '', itemData.get("Movie")) })
    
    rank=getImdbRank({ 'title': itemData['Movie'], 'year':itemData['Year']})
    itemData.update({ 'IMDB_link': rank.get('link'),  'IMDB_rating' : rank.get('rating') })
   # print(itemData)
    return itemData

metaCollections=FillMetaData()
data=pd.merge(data, metaCollections, on='Url', how ='left')

data.to_csv('bollyshare_movies_info.csv', index=False)


FillMetaData Invalid URL 'Frogs-(1972).php': No schema supplied. Perhaps you meant http://Frogs-(1972).php? MissingSchema


[########################################################################] 100%


In [75]:
import requests, bs4 ,re,urllib, pandas as pd

data = pd.read_csv("bollyshare_movies_info.csv") 

filter_m= open("filter_movies.txt", "r")
filter_m=[s.strip().replace('*','') for s in filter_m.readlines()]

filter_movies= data[data["Movie"].str.strip().isin(filter_m)][["Title","Download Full Movie","Download Full HD","IMDB_rating"]]

filter_movies["Download Full HD"]= filter_movies["Download Full HD"].fillna(filter_movies["Download Full Movie"])
filter_movies.drop(['IMDB_rating','Download Full Movie'], axis=1, inplace=True)

#filter_movies.to_csv('bollyshare_movies_filterd.csv', index=False)
filter_movies=filter_movies.reset_index(drop = True) 
print(filter_movies.to_markdown())


|    | Title                    | Download Full HD                                                                                     |
|---:|:-------------------------|:-----------------------------------------------------------------------------------------------------|
|  0 | Sicario (2015)           | https://spaceshut.website/pqhtb8/Sicario-(2015)-hd-download-page.php                                 |
|  1 | Everest (2015)           | https://spaceshut.website/l1vz4p/Everest-(2015)720pfullmovie.php                                     |
|  2 | Lucy (2014)              | https://spaceshut.website/l1vz4p/Lucy-(2014)720pfullmovie.php                                        |
|  3 | Gravity (2013)           | https://spaceshut.website/l1vz4p/Gravity_(2013)fullmovie.php                                         |
|  4 | Shutter Island (2010)    | https://spaceshut.website/l1vz4p/Shutter-Island-(2010)720pfullmovie.php                              |
|  5 | Unknown (2011)           | http://